In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

### Task#1: Merging 12 months data ino a single file

In [ ]:
#to store all the files in a list format
files = [file for file in os.listdir('F:/....../Sales_Data')]

#create a blank dataframe
all_months_data = pd.DataFrame()

#accessing each file one by one from the 'files' list and concatinating it to the blank dataframe created above
for file in files:
    df=pd.read_csv("F:/....../Sales_Data/"+file)
    all_months_data = pd.concat([all_months_data,df])

#exporting the concatenated data to a csv file
#all_months_data.to_csv('F:/....../Sales_Data/All_months_data.csv',index=False)m

#### reading the updated dataframe

In [ ]:
all_data = pd.read_csv('F:/....../Sales_Data/All_months_data.csv')
all_data.head()

### *Question1: Best month for sales? How much was earned that month?*

#### for this task, we need to augment data with addtional columns, such as a 'month' column

### Task#2: Adding 'month' column

#### logic: taking first two characters from the 'order date' column

In [ ]:
all_data['Month'] = all_data['Order Date'].str[0:2]
all_data.head()

#### now, we need to convert this 'Month' column from 'str' to 'int' type

In [ ]:
all_data['Month']=all_data['Month'].astype('int')

#### but, here comes a problem---there are some NaN values in 'Month' column. We need to first address those

### Task#3: Clean up the data

#### drop NaN rows

In [ ]:
#pulling all NaN rows
nan_df = all_data[all_data.isna().any(axis=1)]
nan_df.head()

In [ ]:
all_data = all_data.dropna(how='all')

#### now, we should be able to convert 'Month' column to int

In [ ]:
#still gives error
all_data['Month']=all_data['Month'].astype('int')

#### still, we have some issue with the data---there is some 'or' character in the 'Month' column. This could be because this was present it 'Order Date' column's first two characters

In [ ]:
#finding out where such occurences are coming
temp_df = all_data[all_data['Order Date'].str[0:2]=='Or']
temp_df.head()

#### so, we see that column names are getting duplicated across the dataframe

In [ ]:
#selecting all the records without those rows
all_data = all_data[all_data['Order Date'].str[0:2]!='Or']

#### now, we should be able to convert 'Month' column to int¶

In [ ]:
#it will succeed now
all_data['Month']=all_data['Month'].astype('int')

#### now, continuing with the question at hand- Question1

#### for this, we need to create a new column- sales per order by multiplying quantity and price

### Task#4: Adding a 'sales' column

In [ ]:
all_data.dtypes

#converting the columns to required datatype
#pd.to_numeric is another way of converting to numeric data
all_data['Quantity Ordered'] = pd.to_numeric(all_data['Quantity Ordered'])
all_data['Price Each'] = pd.to_numeric(all_data['Price Each'])

In [ ]:
#creating the sales column
all_data['Sales'] = all_data['Quantity Ordered'] * all_data['Price Each']
all_data.head()

#### now ,returning back to our question- we will be using 'group by' for the same

In [ ]:
#creating a subset of dataframe with the required columns
df_sales = all_data[['Month','Quantity Ordered','Price Each','Sales']]

#applying group by the subset
df_grp=df_sales.groupby('Month').sum()
df_grp

In [ ]:
#alternate solution
months= range(1,13)
plt.bar(months,df_grp['Sales'])
plt.xticks(months)
plt.xlabel('Month no.')
plt.ylabel('Sales in USD (Mn$)')
plt.show()

### *Answer1: December shows the best sales month*

### *Question2: Which city had the highest sales?*

In [ ]:
all_data.head()

#### Task#5: Creating a column-city

In [ ]:
#method1
all_data[['HouseNo_Strt','City1','Country_Pin']] = all_data['Purchase Address'].str.split(",",expand=True)
all_data.head()

In [ ]:
#method2
all_data['City2']=all_data['Purchase Address'].apply(lambda x:x.split(',')[1])

#### we will also need states to uniquely identify cities

In [ ]:
#method3
def getcity(address):
    return address.split(',')[1] #fetching the 1st index item after splitting based on ,

def getstate(address):
    return address.split(',')[2].split(' ')[1] #two levels of splitting is involved here
    #first, splitting based on , and fetching 2nd index value-state+pin
    #second, splitting based on white spaces from the last step data and fetching 1st index because there will be an empty string at th 0th index

all_data['City3']=all_data['Purchase Address'].apply(lambda x:getcity(x) + ' (' + getstate(x) + ')')
all_data.head(1)
    

#### city-wise sales

In [ ]:
df_city= all_data[['City3','Sales']]
df_grp1 = df_city.groupby('City3').sum()
df_grp1

In [ ]:
cities= all_data['City3'].unique()
plt.figure(figsize=(6,3))
plt.bar(cities,df_grp1['Sales'])
plt.xticks(cities,rotation='vertical',size=8)
plt.xlabel('Cities')
plt.ylabel('Sales in USD (Mn$)')

plt.show()

#### but there is a problem with this plot. 
#### acc. to this plot, Austin has the largest sales, but in reality San Fransisco has the largest sales
#### issue is with the order of the data while using the 'unique()' function
#### in y data of plt.bar(cities,df_grp1['Sales']) statement,i.e., df_grp1['Sales'], order matters
#### but, cities= all_data['City3'].unique() statement, orders it in a diff way
#### so, in our plot, x and y are not in the same order
#### below is the correct way

In [ ]:
cities= [city for city, df in all_data.groupby('City3')]
plt.figure(figsize=(6,3))
plt.bar(cities,df_grp1['Sales'])
plt.xticks(cities,rotation='vertical',size=8)
plt.xlabel('Cities')
plt.ylabel('Sales in USD (Mn$)')

plt.show()

### *Answer2: San Fransisco the best sales*

### *Question3: What time should we display ads to maximise likelihood of customer's buying product?* 

In [ ]:
#converting order date to datetime
all_data['Order Date'] = pd.to_datetime(all_data['Order Date'])

In [ ]:
#fetching hour and minute from the 'Order Date' column
all_data['Hour'] = all_data['Order Date'].dt.hour
all_data['Minute'] = all_data['Order Date'].dt.minute

In [ ]:
all_data.head(2)

In [ ]:
hours = [hour for hour,df in all_data.groupby('Hour')]
plt.figure(figsize=(6,2))
plt.plot(hours,all_data.groupby(['Hour']).count())
plt.xticks(hours)
plt.grid()
plt.show()
#all_data.groupby(['Hour']).count()--- this line will give the count of records grouped by hour
#all_data.groupby(['Hour']).count()

### *Answer3: 11AM and 7PM*

### *Question4: What products are most often sold together?* 

In [ ]:
#creating a subset dataframes with only duplicate records
duplicate_df = all_data[all_data['Order ID'].duplicated(keep=False)]
duplicate_df.head(5)

In [ ]:
#creating a single row for each duplciate order id by combining the products together
duplicate_df['Grouped'] = duplicate_df.groupby('Order ID')['Product'].transform(lambda x: ",".join(x))
duplicate_df.head(2)

#### but the above dataframe will still have duplicate rows. we need to drop those duplicate rows

In [ ]:
duplicate_df = duplicate_df[['Order ID','Grouped']].drop_duplicates()
duplicate_df.head(10)

In [ ]:
from itertools import combinations
from collections import Counter

In [ ]:
count = Counter()
for row in duplicate_df['Grouped']:
    row_list= row.split(',')
    count.update(Counter(combinations(row_list,3)))
    
#count variable will have the desired result    

#selecting top 10
for key,value in count.most_common(10):
    print(key,value)


"""
for row in duplicate_df['Grouped']:
    row_list= row.split(',')
    count.update(Counter(combinations(row_list,2)))
    
#count variable will have the desired result    

#selecting top 10
for key,value in count.most_common(10):
    print(key,value)"""

### *Answer4: for a group of two items: ('iPhone', 'Lightning Charging Cable') 1005 and for a group of three items: ('Google Phone', 'USB-C Charging Cable', 'Wired Headphones') 87*

### *Question5: What product sold the most and why?* 

In [ ]:
product_group = all_data.groupby('Product')
#print(product_group)
qty_ordered = product_group['Quantity Ordered'].sum()
qty_ordered

In [ ]:
products = [product for product,df in product_group]
#print(products)
plt.figure(figsize=(4,2))
plt.bar(products,qty_ordered)
plt.xticks(products,rotation='vertical',size=8)
plt.xlabel('Products')
plt.ylabel('Quantity ordered')

plt.show()

### *Answer5(a): AAA Batteries*

#### to know the reason, let's check the correlation of the quantity with price

In [ ]:
prices = product_group['Price Each'].mean()
prices

#### now, we need to overlay this price data in the bar plot

In [ ]:
fig,ax1 = plt.subplots()


ax2 = ax1. twinx()
ax1.bar(products,qty_ordered,color='g')
ax2.plot(products,prices,color='r')

ax1.set_xticklabels(products,rotation='vertical',size=8)
ax1.set_xlabel('Products')
ax1.set_ylabel('Quantity ordered',color='g')
ax2.set_ylabel('Price',color='r')
plt.grid()

plt.show()


### *Answer5(b): whichever product is selling high quantities, have very less price*